In [19]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from src.vectorstore.hybrid_vector_store import HybridVectorStore
from src.config.settings import settings

In [20]:
#### Chroma Vector Store
#### Para criar do zero, ver Vector_Store.ipynb
store = HybridVectorStore(
    persist_path=settings.DATA_DB,
    embedding_model=settings.EMBEDDING_MODEL
)

In [26]:
llm = OllamaLLM(model="LLama3", temperature=0.1)

prompt = ChatPromptTemplate.from_template(
"""
Você é um ACP - um Agente Conversacional Pedagógico especializado em ajudar estudantes com dúvidas relacionadas a conteúdos acadêmicos.
Sua função é fornecer respostas claras e informativas com base no material de estudo fornecido pelo contexto.
Inclua todas as informações relevantes do contexto em suas respostas, evitando suposições ou informações externas.
Suas respostas devem ser sempre em português brasileiro e devem usar um tom leve.

Se não encontrar a resposta no contexto, diga:
"Nenhuma informação disponível no contexto."
            
Contexto:
{context}

Pergunta:
{question}
""")

chain = (
        prompt
        | llm
        | StrOutputParser()
    )


In [29]:
#### Testando prompt com contexto
#### Na teoria, essa resposta será mais embasada nos documentos da RAG
#### PS: Sem GPU leva mais de 1 minuto para responder

def rag_pipeline(question: str):
    docs = store.hybrid_search(question, top_k=5)
    context = "\n\n".join([d[1] for d in docs])

    chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke({"context": context, "question": question})

response = rag_pipeline("Qual a diferença entre Data Scientist e ML Engineer?")
print(response)

Olá!

A pergunta é sobre a diferença entre um Data Scientist e um ML Engineer. De acordo com o contexto, podemos ver que ambos os profissionais trabalham com machine learning, mas têm áreas de foco e especialização diferentes.

Em resumo, as principais diferenças são:

1. Foco: Os Data Scientists se concentram em desenvolver e aprimorar modelos de aprendizado automático, enquanto os ML Engineers se concentracam na engenharia do desenvolvimento, implantação e manutenção desses modelos.
2. Especialização: Os Data Scientists possuem habilidades analíticas e matemáticas fortes, juntamente com conhecimento sobre algoritmos e técnicas de aprendizado automático. Já os ML Engineers têm uma compreensão mais profunda dos princípios de engenharia de software, serviços em nuvem e infraestrutura.
3. Colaboração: Ambas as funções requerem excelência na comunicação e colaboração.

Em resumo, enquanto os Data Scientists se concentram em extrair insights valiosos a partir de grandes volumes de dados, o

In [30]:
#### Testando resposta sem contexto
#### Essa resposta deverá ser mais genérica

def rag_pipeline(question: str):
    # docs = retriever.invoke(question)
    # context = "\n\n".join([d.page_content for d in docs])
    context = ''

    chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke({"context": context, "question": question})

response = rag_pipeline("Qual a diferença entre Data Scientist e ML Engineer?")
print(response)

Olá!

Aqui vai uma resposta baseada no contexto:

Embora ambos os profissionais trabalhem com inteligência artificial (IA) e machine learning (ML), há algumas diferenças importantes entre um Data Scientist e um ML Engineer.

Um Data Scientist é um profissional que se concentra em coletar, processar e analisar grandes volumes de dados para extrair conhecimento valioso. Eles podem trabalhar com diferentes tipos de dados, como texto, imagens ou áudio, e aplicam técnicas estatísticas e de aprendizado automático para identificar padrões e fazer previsões.

Já um ML Engineer é um profissional que se concentra em desenvolver e implementar modelos de aprendizado automático (ML) para resolver problemas específicos. Eles são responsáveis por criar, treinar e depurar modelos de ML para aplicativos como reconhecimento facial, classificação de texto ou predição de comportamentos.

Em resumo, um Data Scientist se concentra em coletar e analisar dados, enquanto um ML Engineer se concentra em desenvol